In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.5.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools PyCall PyPlot Knet HDF5 AutoGrad"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi


echo -e "function KnetArray(x::CuArray{T,N}) where {T,N}\n\tp = Base.bitcast(CuPtr, pointer(x))\n\tk = KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x)\n\tKnetArray{T,N}(k, size(x))\nend" >> /root/.julia/packages/Knet/OYNCT/src/knetarrays/karray.jl

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


# How to run?

First know that this is a one-script notebook to run the entire project. 


1.   Run upper cell. (It install julia 1.5.2)
2.   Restart the notebook. (This allows you to use julia language)
3.   






In [1]:
versioninfo()

Julia Version 1.5.2
Commit 539f3ce943 (2020-09-23 23:17 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


In [ ]:
using Pkg; Pkg.add("AutoGrad")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
Updating `~/.julia/environments/v1.5/Project.toml`
  [6710c13c] + AutoGrad v1.2.3
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [4]:
using Knet
using AutoGrad
using HDF5
using PyCall
using CUDA; aType=(CUDA.functional() ? KnetArray{Float32} : Array{Float32})

KnetArray{Float32,N} where N

In [9]:
CUDA.functional()

true

# DATA

Note that you should upload the DATA.hdf5.gz file to system each time you run, because it is recycled each time.

In [10]:
run(`gunzip DATA.hdf5.gz`)

Process(`gunzip DATA.hdf5.gz`, ProcessExited(0))

In [11]:
fid = h5open("DATA.hdf5", "r")

xTrn = read(fid["trn"]["x"])
xTst = read(fid["tst"]["x"])

yTrn = [read(fid["trn"]["y"][string(i)]) for i in 1:28]
yTst = [read(fid["tst"]["y"][string(i)]) for i in 1:28]

heads = read(fid["heads"])


14-element Array{String,1}:
 "22RV1.AR-C19"
 "22RV1.AR-V7"
 "LNCaP.dht.AR"
 "LNCaP.veh.AR"
 "22RV1.HOXB13"
 "LN95.AR-C19"
 "LN95.AR-V7"
 "LN95.HOXB13"
 "malignant.1.AR"
 "malignant.2.AR"
 "malignant.3.AR"
 "malignant.4.AR"
 "non-malignant.1.AR"
 "non-malignant.2.AR"

In [ ]:
summary.([xTrn, yTrn, xTst, yTst])

4-element Array{String,1}:
 "1000×4×1×13687 Array{Float32,4}"
 "28-element Array{Array{Float32,4},1}"
 "1000×4×1×13688 Array{Float32,4}"
 "28-element Array{Array{Float32,4},1}"

In [20]:
import Pkg; Pkg.test("Knet")

    Testing Knet
Status `/tmp/jl_jq5OHo/Project.toml`
  [6710c13c] AutoGrad v1.2.3
  [052768ef] CUDA v2.3.0
  [5789e2e9] FileIO v1.4.4
  [033835bb] JLD2 v0.2.4
  [1902f260] Knet v1.4.3
  [872c559c] NNlib v0.7.7
  [276daf66] SpecialFunctions v0.10.3
  [8f399da3] Libdl
  [37e2e46d] LinearAlgebra
  [44cfe95a] Pkg
  [de0858da] Printf
  [9a3f8284] Random
  [10745b16] Statistics
  [8dfed614] Test
Status `/tmp/jl_jq5OHo/Manifest.toml`
  [621f4979] AbstractFFTs v0.5.0
  [79e6a3ab] Adapt v2.3.0
  [56f22d72] Artifacts v1.3.0
  [6710c13c] AutoGrad v1.2.3
  [ab4f0b2a] BFloat16s v0.1.0
  [fa961155] CEnum v0.4.1
  [052768ef] CUDA v2.3.0
  [944b1d66] CodecZlib v0.7.0
  [34da2185] Compat v3.23.0
  [e66e0078] CompilerSupportLibraries_jll v0.3.4+0
  [864edb3b] DataStructures v0.18.8
  [e2ba6199] ExprTools v0.1.3
  [5789e2e9] FileIO v1.4.4
  [0c68f7d7] GPUArrays v6.1.2
  [61eb1bfa] GPUCompiler v0.8.3
  [033835bb] JLD2 v0.2.4
  [692b3bcd] JLLWrappers v1.1.3
  [1902f260] Knet v1.4.3
  [929cbde3] LLVM v3.4.

kptr.jl	

┌ Warning: Your Tesla K80 GPU does not meet the minimal required compute capability (3.7.0 < 5.0).
│ Some functionality might not work. For a fully-supported set-up, please use an older version of CUDA.jl
└ @ CUDA ~/.julia/packages/CUDA/YeS8q/src/state.jl:251


kptr:cuda: Test Failed at /root/.julia/packages/Knet/OYNCT/test/kptr.jl:48
  Expression: begin
    p = KnetPtr(128)
    usedmem() == used + 128
end
Stacktrace:
 [1] top-level scope at /root/.julia/packages/Knet/OYNCT/test/kptr.jl:48
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.5/Test/src/Test.jl:1115
 [3] top-level scope at /root/.julia/packages/Knet/OYNCT/test/kptr.jl:48
 34.474069 seconds (51.22 M allocations: 2.737 GiB, 5.28% gc time)
gpu.jl	CUDA toolkit 10.1.243, artifact installation
CUDA driver 10.1.0
NVIDIA driver 418.67.0

Libraries: 
- CUBLAS: 10.2.1
- CURAND: 10.1.1
- CUFFT: 10.1.1
- CUSOLVER: 10.2.0
- CUSPARSE: 10.3.0
- CUPTI: 12.0.0
- NVML: 10.0.0+418.67
- CUDNN: 8.0.4 (for CUDA 10.1.0)
- CUTENSOR: 1.2.1 (for CUDA 10.1.0)

Toolchain:
- Julia: 1.5.2
- LLVM: 9.0.1
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4
- Device support: sm_30, sm_32, sm_35, sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72

┌ Warning: optimizers is deprecated, use sgd, adam etc. instead.
└ @ Knet.Train20 ~/.julia/packages/Knet/OYNCT/src/train20/update.jl:598


...

 62.614463 seconds (82.65 M allocations: 4.192 GiB, 2.70% gc time)
rnn.jl	 70.583695 seconds (53.97 M allocations: 4.658 GiB, 2.69% gc time)
reduction.jl	 66.379151 seconds (108.68 M allocations: 5.208 GiB, 2.68% gc time)
unary.jl	

┌ Warning: invx() is deprecated, please use 1/x instead
└ @ Knet.Ops20 ~/.julia/packages/Knet/OYNCT/src/ops20/activation.jl:61


134.175149 seconds (171.00 M allocations: 8.747 GiB, 2.46% gc time)
binary.jl	151.636173 seconds (197.30 M allocations: 9.668 GiB, 2.99% gc time)
Test Summary:     | Pass  Fail  Error  Broken  Total
Knet              | 9356    13     12       5   9386
  kptr:alloc      |    8                           8
  kptr:gc         |    7                           7
  kptr:realloc    |    8                           8
  kptr:cuda       |          1                     1
  kptr:cudagc     |    1                           1
  gpu             |   14                          14
  distributions   |    4                           4
  dropout         |    3                           3
  gcnode          |    5                           5
  JLD             |    4                           4
  statistics      |   68                          68
  bmm             |   39                          39
  serialize       |   19                          19
  loss            |  143                         143
  cuar

ERROR: LoadError: Some tests did not pass: 9356 passed, 13 failed, 12 errored, 5 broken.
in expression starting at /root/.julia/packages/Knet/OYNCT/test/runtests.jl:7


LoadError: ignored

In [ ]:
%%shell
set -e

read -r -d '' FIX <<- EOM
    function KnetArray(x::CuArray{T,N}) where {T,N}
        p = Base.bitcast(CuPtr, pointer(x))
        k = KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x)
        KnetArray{T,N}(k, size(x))
    end
EOM

echo $FIX

In [145]:
run(```
ls /root/.julia/packages/Knet/OYNCT/src/knetarrays/karray.jl
```)
#run(`FIX="function KnetArray(x::CuArray{T,N}) where {T,N}; p = Base.bitcast(CuPtr, pointer(x)); k = KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x); KnetArray{T,N}(k, size(x));  end"`)

run(`echo "function KnetArray(x::CuArray{T,N}) where {T,N}" >> /root/.julia/packages/Knet/OYNCT/src/knetarrays/karray.jl`)
run(`echo "\tp = Base.bitcast(CuPtr, pointer(x))" >> /root/.julia/packages/Knet/OYNCT/src/knetarrays/karray.jl`)
run(`echo "\tk = KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x)" >> /root/.julia/packages/Knet/OYNCT/src/knetarrays/karray.jl`)
run(`echo "\tKnetArray{T,N}(k, size(x))" >> /root/.julia/packages/Knet/OYNCT/src/knetarrays/karray.jl`)
run(`echo "end" >> /root/.julia/packages/Knet/OYNCT/src/knetarrays/karray.jl`)






/root/.julia/packages/Knet/OYNCT/src/knetarrays/karray.jl


LoadError: ignored

In [144]:
run(``` \
echo "\
    function KnetArray\(x\:\:CuArray\{T,N\}) where \{T,N\} \
        p = Base.bitcast\(CuPtr, pointer\(x\)\) \
        k = KnetPtr\(p, sizeof\(x\), Int\(CUDA.device\(\).handle\), x\) \
        KnetArray\{T,N\}\(k, size\(x\)\) \
    end " 
>> /root/.julia/packages/Knet/OYNCT/src/knetarrays/karray.jl
```)

LoadError: ignored

In [ ]:
 >> /root/.julia/packages/Knet/src/knetarrays/karray.jl

In [46]:


function KnetArray(x::CuArray{T,N}) where {T,N}
    p = Base.bitcast(Cptr, pointer(x))
    k = KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x)
    KnetArray{T,N}(k, size(x))
end


aType = Array{Float32}
function minibatch(
    x, y; batchsize::Int32=100, shuffle::Bool=false, atype::Type=Array{Float32})
    """
    This function converts x and y data. Then initiates and returns Data type object.
    """
    etype = eltype(atype)
    x = aType(x)
    y = [atype(yi) for yi in y]
    n = size(x)[end]
    Data(x,y, batchsize, shuffle, n, 1:n)
end


mutable struct Data
    x
    y
    batchsize::Int32
    shuffle::Bool
    n::Int32
    indices::Vector{Int32}
end

import Base: length, iterate, eltype, HasEltype

function length(d::Data)
    # Your code here
    Int(ceil(d.n / d.batchsize))
end



function iterate(d::Data, state=(0)) # here the start point of state is 0
    if state == 0 && d.shuffle
        d.indices = randperm(d.n)
    end

    if state >= d.n
        return nothing
    end
    i = state + 1 # here the beginning of the current slice
    j = state + d.batchsize # here the end of the current slice

    idx = d.indices[i:j]
    xbatch = d.x[:,:,:,i:j]
    ybatch = [y[:,:,:,i:j] for y in d.y]

    return ((xbatch, ybatch), j) # here it returns the batch and the next state of the iteration
end



iterate (generic function with 337 methods)

In [47]:
dTrn = minibatch(xTrn, yTrn; batchsize=Int32(100), atype=aType);
dTst = minibatch(xTst, yTst; batchsize=Int32(100), atype=aType);

In [48]:
first(dTrn)

(Float32[0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]

Float32[0.0 0.0 0.0 1.0; 0.0 1.0 0.0 0.0; … ; 0.0 1.0 0.0 0.0; 1.0 0.0 0.0 0.0]

Float32[0.0 0.0 0.0 1.0; 0.0 0.0 0.0 1.0; … ; 0.0 0.0 1.0 0.0; 0.0 1.0 0.0 0.0]

...

Float32[1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0; … ; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]

Float32[0.0 0.0 0.0 1.0; 1.0 0.0 0.0 0.0; … ; 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0]

Float32[0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; … ; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0], Array{Float32,4}[[0.0; 0.0; … ; 0.0; 0.0]

[0.0; 0.0; … ; 0.0; 0.0]

[0.0; 0.0; … ; 0.0; 0.0]

[0.0; 0.0; … ; 0.0; 0.0]

[0.0; 0.0; … ; 0.0; 0.0]

[0.0; 0.0; … ; 0.0; 0.0]

...

[0.0; 0.0; … ; 0.0; 0.0]

[0.0; 0.0; … ; 0.0; 0.0]

[0.0; 0.0; … ; 0.0; 0.0]

[0.0; 0.0; … ; 0.0; 1.07893]

[0.0; 0.0; … ; 0.0; 0.0]

[0.0; 0.0; … ; 0.0; 0.0], [30.210041]

[29.131111]

[5.39465]

[4.31572]

[20.499672]

[19.420742]

...

[18.341812]

[20.499672]

[664.6213]

[663.5421]

[12.947161]

[10.789301], [0.0; 0.0; … ; 

# Losses

In [49]:


function aproxLogFact(n)
    """
    Approximation for log n!
    """
    Float32(n * log(n) - n  + log(n * (1 + 4n * (1 + 2n)))/6 + log(π)/2)
end


function multinomial_nll(ypred, ytrue)
    pV = ypred ./ sum(ypred, dims=1)
    logPV = log.(pV)
    pS = sum(logPV .* ytrue, dims=1)
    n = sum(ytrue, dims=1)
    nS = (aproxLogFact.(n) .- sum(aproxLogFact.(ytrue), dims=1))
    logP = pS .+ nS
    loss = sum(logP) / prod(size(logP))
end

function mse(ypred, ytrue)
    λ = Float32(0.5) * sum(ypred) / size(ypred, 1)
    C = λ * abs2.(log.(1 .+ sum(ypred)) .- log.(1 .+ ytrue)) ./ size(ypred, 1)
    sum(C) / prod(size(C))
end


mse (generic function with 1 method)

# Layers

In [126]:


mutable struct Conv
    w
    b
    f
    pDrop
    padding
    dilation
    nParameters
end



(c::Conv)(x) =
    c.f.(
        conv4(
            c.w,
            dropout(x, c.pDrop),
            padding=c.padding,
            dilation=c.dilation
        ) .+ c.b
    )



Conv(w1::Int, w2::Int, cx::Int, cy::Int;
    f=
        relu,
    pDrop=
        0,
    padding=
        0,
    dilation=
        1,
    xType=
        Array{Float32},
    scale=
        0.01
    ) = Conv(
            Param(xType(rand(w1,w2,cx,cy)) .* eltype(xType)(scale), Adam()),
            Param(xType(zeros(1,1,cy,1)), Adam()),
            f,
            pDrop,
            padding,
            dilation,
            (w1*w2*cx +1)*cy + cy
        )

################################################################################
struct Pool
    wSize
    nParameters
end

(p::Pool)(x) = pool(x; window=p.wSize, mode=1)
Pool(x) = Pool(x, 0)
################################################################################
mutable struct BatchNorm
    moments
    params
    nParameters
end

(b::BatchNorm)(x) = batchnorm(x, b.moments, b.params)

BatchNorm(C) = BatchNorm(bnmoments(), arrayType(bnparams(C)), 0)

################################################################################

struct Dense
    w
    b
    f
    pDrop
    window
    nParameters
end

Dense(i,o;
    f=
        identity,
    window=
        1,
    xType=
        Array{Float32},
    scale=
        0.01,
    pDrop=
        0
    ) =
        Dense(
            Param(xType(rand(o,i)) .* eltype(xType)(scale), Adam()),
            Param(xType(zeros(o)), Adam()),
            f,
            pDrop,
            window,
            (i+1)*o + o
        )

(d::Dense)(x) =
    d.f.(
        d.w *
            mat(
                dropout(x, d.pDrop),
            ) .+ d.b
        )


################################################################################


# Define a deconvolution layer:

struct DeConv
    w
    b
    padding
    nParameters
end


DeConv(w1::Int, w2::Int, cx::Int, cy::Int;
    padding=
        0,
    xType=
        Array{Float32},
    scale=
        0.01
    ) = DeConv(
            Param(xType(rand(w1,w2,cx,cy)) .* eltype(xType)(scale), Adam()),
            Param(xType(zeros(1,1,cx,1)), Adam()),
            padding,
            (w1*w2*cx)*cy + cx
        )



(dC::DeConv)(x) =
    deconv4(
        dC.w,
        x,
        padding=dC.padding
    ) .+ dC.b
################################################################################


# Architecture

In [51]:

# Define a body (up to bottleneck)
mutable struct Body
    layers
    Body(layers...) = new(layers)
end


function addLayer!(b::Body, l)
    b.layers = (
        b.layers...,
        l
    )
end


(b::Body)(x) = (for l in b.layers;x = l(x);end;x)


##################################

# Define Head structure

mutable struct Head
    layers
    lossF
    Head(layers...; lossF=identity) = new(layers, lossF)
end

(h::Head)(x) = (for l in h.layers;x = l(x);end;x)



###########################


mutable struct Chain
    body
    heads
    n
    bottleneck
    Chain(body, heads...; n=0, bottleneck=nothing) = new(body, heads..., n, bottleneck)
end

Chain(b::Body) = Chain(b, []; n=0)
(c::Chain)(h::Head...) = addHead!(c, h...)

function addHead!(c::Chain, h...)
    c.heads = vcat(c.heads..., h...)
    c.n = length(c.heads)
end

##########################################


function meanLoss(c::Chain, d::Data)
    J = [Float32(0) for n in 1:c.n]
    for (j, (x, y)) in enumerate(d)
        for (hIdx, (h, yi)) in enumerate(zip(c.heads, y))
            J[hIdx] += h.lossF(h(c.body(x)), yi)
        end
    end
    J ./ length(d)
end


every(n,itr) = (x for (i,x) in enumerate(itr) if i%n == 0)


every (generic function with 1 method)

In [52]:

function train!(c::Chain, dTrn::Data, dTst::Data; iters=500, period=10)
    results = []
    for i=1:period:iters
        push!(
            results, (
                c(dTrn),
                c(dTst),
                1-accuracy(c; data=dTrn),
                1-accuracy(c; data=dTst)
            )
        )
        for (x, y) in every(period, dTrn)
            J = @diff c(x, y)
            for p in params(J)
                ∇p = grad(J, p)
                update!(p, ∇p)
            end
        end
    end
    push!(
        results, (
            c(dTrn),
            c(dTst),
            1-accuracy(c; data=dTrn),
            1-accuracy(c; data=dTst)
        )
    )
    results = reshape(collect(Float32,flatten(results)),(4,:))
    return 0:period:iters, results
end


train! (generic function with 1 method)

In [53]:

function meanLoss(c::Chain, d::Data)
    J = [Float32(0) for n in 1:c.n]
    for (j, (x, y)) in enumerate(d)
        for (hIdx, (h, yi)) in enumerate(zip(c.heads, y))
            J[hIdx] += h.lossF(h(c.body(x)), yi)
        end
    end
    J ./ length(d)
end


function meanAccuracy(c::Chain, d::Data)
    A = [Float32(0) for n in 1:c.n]
    for (j, (x, y)) in enumerate(d)
        for (hIdx, (h, yi)) in enumerate(zip(c.heads, y))
            A[hIdx] += sum(h(c.body(x)) .== yi)
        end
    end
    A ./ length(d)
end


meanAccuracy (generic function with 1 method)

In [125]:
meanLoss(bpnet, dTrn)

LoadError: ignored

# Model

In [123]:

seqLen = size(xTrn, 1)

tasks = [
  "22RV1.AR-C19", "22RV1.AR-V7", "LNCaP.dht.AR", "LNCaP.veh.AR", "22RV1.HOXB13", 
  "LN95.AR-C19", "LN95.AR-V7", "LN95.HOXB13", "malignant.1.AR", "malignant.2.AR",
  "malignant.3.AR", "malignant.4.AR", "non-malignant.1.AR", "non-malignant.2.AR"
]


bpnetBody = Body(
    Conv(25,4,1,64, padding=(12, 0)) # Int(max((cx - 1) * s1 + w1 - cx, 0)/2)
)



nLayer = 2

for i=1:nLayer
    rate = 2^i
    addLayer!(bpnetBody,
        Conv(3,1,64,64, padding=(rate, 0), dilation=(rate, 0))
    )
end


bpnet = Chain(bpnetBody)


for task in tasks
    profileHead = Head(
        DeConv(25,1,2,64, padding=(12,0));
        lossF=
            multinomial_nll
    )
    println("profileHead")
    countHead = Head(
        Pool((seqLen, 1)),
        Dense(64,2, window=(seqLen,1));
        lossF=
            mse
    )
    println("countHead")
    bpnet(profileHead, countHead)
end




profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead
profileHead
countHead


In [56]:

[l.nParameters for l in collect(bpnet.body.layers)]
[l.nParameters for h in collect(bpnet.heads) for l in h.layers]


28-element Array{Int64,1}:
 3200
  130
 3200
  130
 3200
  130
 3200
  130
 3200
  130
 3200
  130
 3200
    ⋮
 3200
  130
 3200
  130
 3200
  130
 3200
  130
 3200
  130
 3200
  130

In [ ]:
training = train!(bpnet, dTrn, dTst)
